In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
! pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 75.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 29.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 99.3 MB/s eta 0:00:00


In [ ]:
#импорты бибилиотек
import warnings

import torch
import pandas as pd
import numpy as np
from joblib import dump, load
from transformers import BertForSequenceClassification, BertTokenizer
from tqdm import notebook
from tqdm.notebook import tqdm

warnings.filterwarnings('ignore')

In [ ]:
#Константы
RANDOM_STATE = 42
TEST_SIZE = 0.25
MAX_LENGTH = 512

https://drive.google.com/file/d/1AcgkP3Dzmc-kfU2NAvgoyAtAsxHBirYQ/view?usp=sharing

In [ ]:
# Загружаем файл с reviews с диска
! gdown 1AcgkP3Dzmc-kfU2NAvgoyAtAsxHBirYQ

Downloading...
From: https://drive.google.com/uc?id=1AcgkP3Dzmc-kfU2NAvgoyAtAsxHBirYQ
To: /content/reviews_cleared.csv
100% 88.9M/88.9M [00:03<00:00, 24.6MB/s]


In [ ]:
df = pd.read_csv('/content/reviews_cleared.csv')
df.head()

,listing_id,comments
0,882380,"Safia's apartments are gorgeous! Brand new, im..."
1,882380,Safia is an amazing hostess. She is attentive ...
2,882380,The host canceled this reservation 2 days befo...
3,1665188,My family had a fantastic stay at the beautifu...
4,2617879,Rob was very helpful and pleasant prior to our...


In [ ]:
len(df)

346556

In [ ]:
# Проверка на дубликаты
df.duplicated().sum()

299

In [ ]:
# Удаление дубликатов
df = df.drop_duplicates()
df.duplicated().sum()

0

In [ ]:
# Определим максимальную длину комментария
max([len(com) for com in df['comments']])

5378

In [ ]:
# Загрузка предобученной модели BERT и токенайзера

model = BertForSequenceClassification.from_pretrained('bert-base-multilingual-cased', num_labels=5)
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model ch

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else "cpu")
torch.device('cuda')

device(type='cuda')

In [ ]:
tqdm.pandas()
tokenized = df['comments'].progress_apply(
    lambda x: tokenizer.encode(x, max_length=MAX_LENGTH, truncation=True, add_special_tokens=True))

padded = np.array([i + [0]*(MAX_LENGTH - len(i)) for i in tokenized.values])

attention_mask = np.where(padded != 0, 1, 0)

  0%|          | 0/346257 [00:00<?, ?it/s]

In [ ]:
len(tokenized)

346257

In [ ]:
# Сохраняем промежуточные результаты
dump(tokenized, "/content/drive/MyDrive/Data sciense/Kaggle/School_21_competition/tokenized.joblib")
dump(padded, "/content/drive/MyDrive/Data sciense/Kaggle/School_21_competition/padded.joblib")
dump(attention_mask, "/content/drive/MyDrive/Data sciense/Kaggle/School_21_competition/attention_mask.joblib")

['/content/drive/MyDrive/Data sciense/Kaggle/School_21_competition/attention_mask.joblib']

In [ ]:
load("/content/drive/MyDrive/Data sciense/Kaggle/School_21_competition/tokenized.joblib")

0         [101, 12404, 29347, 112, 187, 103810, 10301, 1...
1         [101, 12404, 29347, 10124, 10151, 28149, 19308...
2         [101, 10117, 19317, 10944, 32942, 10336, 10531...
3         [101, 11590, 11365, 10374, 169, 69262, 97656, ...
4         [101, 20386, 10134, 12558, 15217, 14446, 10111...
                                ...                        
346551    [101, 12489, 169, 43477, 12566, 170, 10115, 10...
346552    [101, 79657, 117, 32650, 28841, 13499, 117, 64...
346553    [101, 10117, 19317, 10944, 32942, 10336, 10531...
346554    [101, 10117, 19317, 10944, 32942, 10336, 10531...
346555    [101, 10117, 19317, 10944, 32942, 10336, 10531...
Name: comments, Length: 346257, dtype: object

In [ ]:
tokenized[:1000]

0      [101, 12404, 29347, 112, 187, 103810, 10301, 1...
1      [101, 12404, 29347, 10124, 10151, 28149, 19308...
2      [101, 10117, 19317, 10944, 32942, 10336, 10531...
3      [101, 11590, 11365, 10374, 169, 69262, 97656, ...
4      [101, 20386, 10134, 12558, 15217, 14446, 10111...
                             ...                        
995    [101, 9357, 70450, 11467, 9099, 8942, 118707, ...
996    [101, 10631, 10134, 169, 69262, 97656, 19317, ...
997    [101, 12592, 18214, 10106, 169, 14772, 11192, ...
998    [101, 10747, 11192, 10124, 48573, 10454, 28780...
999    [101, 63537, 13684, 54617, 10169, 169, 11367, ...
Name: comments, Length: 1000, dtype: object

In [ ]:
batch_size = 1
preds = []

for i in notebook.tqdm(range(padded[:1000].shape[0] // batch_size)):
        batch = torch.LongTensor(padded[:1000][batch_size*i:batch_size*(i+1)]).to(device)
        attention_mask_batch = torch.LongTensor(attention_mask[:1000][batch_size*i:batch_size*(i+1)]).to(device)
        
        with torch.no_grad():
            model = model.to(device)
            batch_embeddings = model(batch, attention_mask=attention_mask_batch)
            logits = batch_embeddings[0]
            probabilities = torch.softmax(logits, dim=1)
            predictions = torch.argmax(probabilities, dim=1)
        
        preds.append(predictions)
        dump(preds, "/content/drive/MyDrive/Data sciense/Kaggle/School_21_competition/preds.joblib")

  0%|          | 0/1000 [00:00<?, ?it/s]

In [ ]:
len(preds)

1000

In [ ]:
t = df[:1000]
t

,listing_id,comments
0,882380,"Safia's apartments are gorgeous! Brand new, im..."
1,882380,Safia is an amazing hostess. She is attentive ...
2,882380,The host canceled this reservation 2 days befo...
3,1665188,My family had a fantastic stay at the beautifu...
4,2617879,Rob was very helpful and pleasant prior to our...
...,...,...
995,13574665,복층으로 된 깔끔하고 아름다운 집입니다. 쇼디치와 가깝고 주변이 모두 집이라 안전합...
996,13574665,"Michael was a fantastic host, quick to answer,..."
997,13574665,Great location in a great place.. Michael was ...
998,13574665,This place is absolutely stunningly designed. ...


In [ ]:
for i in range(len(preds)):
  preds[i] = preds[i].cpu().numpy()

In [ ]:
t['ratings'] = preds
t[['comments', 'ratings']].head(15)
#embeddings.append(batch_embeddings[0][:,0,:].cpu().numpy())

,comments,ratings
0,"Safia's apartments are gorgeous! Brand new, im...",[3]
1,Safia is an amazing hostess. She is attentive ...,[3]
2,The host canceled this reservation 2 days befo...,[1]
3,My family had a fantastic stay at the beautifu...,[3]
4,Rob was very helpful and pleasant prior to our...,[3]
5,Inge is a lovely lady she will definitely make...,[3]
6,Expect a luxurious accommodation and a very wa...,[3]
7,"Inge, Matt's mum is an amazing woman. She made...",[3]
8,"I only staid one night, but the room is fantas...",[3]
9,"Esperienza fantastica: la casa era bellissima,...",[3]


In [ ]:
pd.set_option('display.max_colwidth', 200)

In [ ]:
t.loc[t['ratings']==1].head()

,listing_id,comments,ratings
2,882380,The host canceled this reservation 2 days before arrival. This is an automated posting.,[1]
28,2926013,Thank you for your nice welcoming! the room was exactly what i was expecting and the bed was really comfortable. Il i ever comme back to London i will definitely come and sleep at your nice house....,[1]
31,2926013,it was very nice to stay here. the landlord is very very kind and friendly.i and my son just been here for six days and he still brought the small bed for my little boy!that was so sweet. i’ll mis...,[1]
37,2926013,"This was my second time staying, and I had a great experience. The location of the property was perfect, Matt and Inge were great hosts, and it was great value for the money.",[1]
47,2926013,"War wunderschön, wir würden jederzeit wieder hin.",[1]


In [ ]:
t[['comments', 'ratings']].tail(15)

,comments,ratings
985,Close to transport and stores! Nice couple and clean room.,[3]
986,"Big room, big bed, and a sofa. I felt very welcomed by Izy & Kisler. Everything was very clean and confortable. I recommend it without a doubt.",[3]
987,"Izy and Kisler live close to a tube station (a less than five minutes' walk), in a quiet residential area. Their guest room is pretty big, with a large bed and a comfy sofa. They seemed eager to m...",[3]
988,"Nice clean room, clean laundry and comfortable double bed. Clean bathroom and good shower, and nice kitchen. I would not advise arriving after dark, however, if you are a lone female as the walk f...",[3]
989,U can expect a very friendly and hospitable welcome and everything of necessities at your disposal. I had a great experience :),[3]
990,Stayed there twice for a total of 5 weeks. Very nice n clean and the host is very helpful and flexible,[3]
991,"J'ai été dans ce Airbnb pendant 2 mois et 2 semaines du 09 Avril au 15 juin 2018. Les hôtes étant une famille, j'ai été accueilli par un homme (sa femme étant absente). Un peu gênée par le fait qu...",[3]
992,it was a vry gud experience...harold was a great host....was attending common station academy and stayed here for 5 weeks with my wife...its vry close to acadmy...only 4 mints walk...had huge adva...,[2]
993,I really liked the fact that this was right next to common stations just 5mins walk but it was too small for me and my husband to stay in for a period of 15 days. At 1 point of time there were abo...,[3]
994,Great located superbe and friendly neighborhood in Islington. Everything went smoothly and very welcoming with Michael.,[3]


In [ ]:
send_telegram('Код выполнен!')

NameError: ignored